In [18]:
import pandas as pd
import numpy as np
import json
import requests

In [19]:
import sys
sys.path.append('../')

In [20]:
def GetPosition (array):
    if isinstance(array,np.ndarray):
        if array.__contains__('C'):
            return 'C'
        if array.__contains__('2B'):
            return '2B'
        if array.__contains__('3B'):
            return '3B'
        if array.__contains__('SS'):
            return 'SS'
        if array.__contains__('1B'):
            return '1B'
        if array.__contains__('OF'):
            return 'OF'
        else:
            return 'Util'
    else: 
        return 'Util'

In [21]:
atcpit=pd.read_json('https://www.fangraphs.com/api/projections?stats=pit&type=atc')
atcbat=pd.read_json('https://www.fangraphs.com/api/projections?stats=bat&type=atc')

In [22]:
atcbat

,Name,Team,ShortName,G,AB,PA,H,1B,2B,3B,...,IntraSD,ADP,Pos,minpos,teamid,League,PlayerName,playerids,playerid,.
0,"<a href=""statss.aspx?playerid=18401&position=O...",ATL,Braves,147.5230,566.9760,659.7560,177.1620,106.57900,32.99280,2.301720,...,1.161340,1.030000,0.012658,OF,16,NL,Ronald Acuña Jr.,18401,18401,
1,"<a href=""statss.aspx?playerid=13611&position=O...",LAD,Dodgers,149.1070,565.7660,660.7330,160.3420,90.02950,35.35570,2.136330,...,0.865364,4.970000,0.000000,OF,22,NL,Mookie Betts,13611,13611,
2,"<a href=""statss.aspx?playerid=15640&position=O...",NYY,Yankees,137.8050,495.1490,598.9940,136.9190,71.51120,21.62790,0.305957,...,1.389700,12.310000,0.345455,OF,9,AL,Aaron Judge,15640,15640,
3,"<a href=""statss.aspx?playerid=20123&position=O...",NYY,Yankees,152.7720,528.6340,663.5400,147.8860,82.76200,26.81160,1.558040,...,0.962505,10.280000,0.037500,OF,9,AL,Juan Soto,20123,20123,
4,"<a href=""statss.aspx?playerid=19556&position=D...",HOU,Astros,137.2960,499.9400,592.6580,148.1720,77.66480,30.49940,1.691100,...,1.473900,15.990000,0.646018,DH/OF,21,AL,Yordan Alvarez,19556,19556,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,"<a href=""statss.aspx?playerid=12981&position=C...",PHI,Phillies,33.8422,88.1550,95.5001,16.4555,11.76640,3.61850,0.088329,...,1.641870,999.000000,0.000000,C,26,NL,Cam Gallagher,12981,12981,
683,"<a href=""statss.aspx?playerid=14196&position=S...",ATL,Braves,30.5814,67.2710,74.1473,13.8212,9.46698,2.55229,0.216475,...,1.859520,999.000000,0.000000,SS,16,NL,Andrew Velazquez,14196,14196,
684,"<a href=""statss.aspx?playerid=25845&position=1...",COL,Rockies,31.4182,109.4470,121.0300,23.8653,14.34810,5.18995,0.596825,...,1.528250,727.989990,0.000000,1B/OF,19,NL,Michael Toglia,25845,25845,
685,"<a href=""statss.aspx?playerid=23695&position=O...",NYY,Yankees,18.3067,59.7153,65.0526,12.7788,9.04195,2.16384,0.121544,...,1.787700,743.349976,0.000000,OF,9,AL,Everson Pereira,23695,23695,


In [23]:
hittersmean = atcbat
pitchersmean = atcpit

In [24]:
hittersmean['OBPOnTeam'] = (hittersmean['PA']*hittersmean['OBP']+2037.4)/(hittersmean['PA']+6100)
hittersmean['SLGOnTeam'] = (hittersmean['AB']*hittersmean['SLG']+2420)/(hittersmean['AB']+5500)
hittersmean['OPSValue'] = (hittersmean['OBPOnTeam']+hittersmean['SLGOnTeam']-.774)/.006

hittersmean['AVGValue'] = ((hittersmean['H']+1768)/(hittersmean['AB']+6100)-.267)/.0024
hittersmean['value'] = hittersmean['OPSValue']+(hittersmean['R']/8.92+hittersmean['HR']/7.5+hittersmean['RBI']/8.92+hittersmean['SB']/6.0)

In [25]:
import pybaseball
positions = pybaseball.fielding_stats(2023, qual=0)

In [26]:
positions['Pos']=positions['Pos'].replace(['LF','CF','RF'],'OF')


In [27]:
positions_unique=positions.loc[(positions['G']>=10) | (positions['GS']>=5)].groupby('IDfg')['Pos'].unique()


In [28]:
positions

,IDfg,Season,Name,Team,Pos,G,GS,Inn,PO,A,...,60-90%,# 60-90%,90-100%,# 90-100%,rSZ,rCERA,rTS,FRM,OAA,RAA
0,27478,2023,Patrick Bailey,SFG,C,94,84,765.2,740,50,...,NaN,NaN,NaN,NaN,11.0,1.0,NaN,17.4,NaN,NaN
1,16930,2023,Jonah Heim,TEX,C,124,116,993.1,938,40,...,NaN,NaN,NaN,NaN,4.0,-1.0,NaN,9.3,NaN,NaN
2,21534,2023,Cal Raleigh,SEA,C,128,114,1038.0,1083,40,...,NaN,NaN,NaN,NaN,5.0,-6.0,NaN,8.4,NaN,NaN
3,25479,2023,Brenton Doyle,COL,OF,125,118,1023.2,373,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0
4,20503,2023,William Contreras,MIL,C,108,108,942.2,969,30,...,NaN,NaN,NaN,NaN,9.0,-1.0,NaN,14.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,31533,2023,Victor Mederos,LAA,P,3,0,3.0,0,0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
2289,31757,2023,Mason Miller,OAK,P,10,6,33.1,2,1,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-0.4,NaN,NaN
2290,31776,2023,Richard Kerkering,PHI,P,3,0,3.0,0,0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-0.1,NaN,NaN
2291,31838,2023,Kodai Senga,NYM,P,29,29,166.1,14,8,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.7,NaN,NaN


In [29]:
positions_unique.index=positions_unique.index.map(str)

In [30]:
hitterspos=pd.merge(hittersmean,positions_unique,how='left',left_on='playerids',right_index=True)

In [31]:
hitterspos['Pos']=hitterspos['Pos_y']

In [32]:
hitterspos['BestPos']=hitterspos['Pos'].apply(GetPosition)

hitterspos['rank']=hitterspos.groupby('BestPos')['value'].rank(ascending=False)
hitterspos['utilrank']=hitterspos['value'].rank(ascending=False)

###customized league replacement value
replacementvalue=hitterspos.groupby('BestPos').value.nlargest(8).groupby('BestPos').min().reset_index(name='PosValue')
replacementvalue['UtilValue']=float(hitterspos.loc[hitterspos['utilrank']==64]['value'])

replacementvalue['ReplacementValue']=pd.DataFrame([replacementvalue['UtilValue'], replacementvalue['PosValue']]).min()
#replacementvalue['ReplacementValue']=replacementvalue['UtilValue']
replacementvalue.loc[replacementvalue['BestPos'] == 'Util', 'ReplacementValue'] = replacementvalue['UtilValue']

/var/folders/k3/m2tddk1x3fvd5dtgwd9h1jcr0000gp/T/ipykernel_75327/3997149529.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  replacementvalue['UtilValue']=float(hitterspos.loc[hitterspos['utilrank']==64]['value'])


In [33]:
hittersmean[hittersmean['PlayerName']=='Bryce Harper']

,Name,Team,ShortName,G,AB,PA,H,1B,2B,3B,...,League,PlayerName,playerids,playerid,.,OBPOnTeam,SLGOnTeam,OPSValue,AVGValue,value
25,"<a href=""statss.aspx?playerid=11579&position=O...",PHI,Phillies,145.441,527.861,627.713,150.276,86.0916,33.5366,1.46418,...,NL,Bryce Harper,11579,11579,,0.339148,0.446973,2.020134,9.344211,28.902903


In [34]:
replacementvalue

,BestPos,PosValue,UtilValue,ReplacementValue
0,1B,24.290961,21.694032,21.694032
1,2B,22.042845,21.694032,21.694032
2,3B,23.953516,21.694032,21.694032
3,C,18.217614,21.694032,18.217614
4,OF,30.708992,21.694032,21.694032
5,SS,23.514326,21.694032,21.694032
6,Util,17.353118,21.694032,21.694032


In [35]:
hittersfinal=hitterspos.reset_index().merge(replacementvalue,on='BestPos',how='left')

hittersfinal['VORP']=hittersfinal['value']-hittersfinal['ReplacementValue']


In [36]:
pitchersmean.columns

Index(['Name', 'Team', 'ShortName', 'W', 'L', 'GS', 'G', 'SV', 'HLD', 'BS',
       'IP', 'TBF', 'H', 'R', 'ER', 'HR', 'SO', 'BB', 'IBB', 'HBP', 'ERA',
       'WHIP', 'K/9', 'BB/9', 'K/BB', 'HR/9', 'K%', 'BB%', 'K-BB%', 'GB%',
       'AVG', 'BABIP', 'LOB%', 'FIP', 'WAR', 'RA9-WAR', 'InterSD', 'InterSK',
       'IntraSD', 'QS', 'ADP', 'teamid', 'League', 'PlayerName', 'playerids',
       'playerid', '.'],
      dtype='object')

In [37]:

########################
#CHANGE THE COLUMNS BELOW BASED ON SGP VALUES FOR YOUR LEAGUE
#I think that you can use SGP based on MLB league averages
pitchersmean['ERA']=(525.55+pitchersmean['ER'])*9/(1350.0+pitchersmean['IP'])/-0.106
pitchersmean['WHIP'] =(1620.0+pitchersmean['H']+pitchersmean['BB'])/(1350.0+pitchersmean['IP'])/-0.02
pitchersmean['KBB'] = ((55*9+9*pitchersmean['K/9'])/(55*3+pitchersmean['BB']))/0.09
pitchersmean['value'] = pitchersmean['KBB']+pitchersmean['ERA']+pitchersmean['WHIP']+(pitchersmean['W']/4.05+(pitchersmean['SV']+pitchersmean['HLD'])/10.0)
########################

pitchersmean['utilrank']=pitchersmean['value'].rank(ascending=False)

###customized league replacement value
pitchersmean['ReplacementValue']=float(pitchersmean.loc[pitchersmean['utilrank']==80]['value'])

pitchersmean['VORP']=pitchersmean['value']-pitchersmean['ReplacementValue']


/var/folders/k3/m2tddk1x3fvd5dtgwd9h1jcr0000gp/T/ipykernel_75327/4235124889.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pitchersmean['ReplacementValue']=float(pitchersmean.loc[pitchersmean['utilrank']==80]['value'])


In [38]:
pitchersmean['BB']

0      59.15580
1      44.49430
2      41.06190
3      47.01950
4      41.11040
         ...   
912    10.27900
913    10.62370
914     3.76392
915    26.00430
916    34.37570
Name: BB, Length: 917, dtype: float64

In [39]:
final=pd.concat([hittersfinal,pitchersmean])


In [40]:
final['VORP']=final['VORP'].round(2)

In [46]:
final['PlayerName']=final['PlayerName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [41]:
final.sort_values('VORP', ascending=False)

,index,Name,Team,ShortName,G,AB,PA,H,1B,2B,...,BB/9,K/BB,HR/9,K-BB%,GB%,LOB%,FIP,RA9-WAR,QS,KBB
0,0.0,"<a href=""statss.aspx?playerid=18401&position=O...",ATL,Braves,147.52300,566.9760,659.7560,177.16200,106.57900,32.992800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,16.0,"<a href=""statss.aspx?playerid=19755&position=D...",LAD,Dodgers,139.36200,514.9280,608.3170,144.37200,74.67980,26.864100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10.0,"<a href=""statss.aspx?playerid=25764&position=S...",KCR,Royals,152.02900,605.5440,658.3310,169.38900,99.91130,33.281300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,"<a href=""statss.aspx?playerid=15640&position=O...",NYY,Yankees,137.80500,495.1490,598.9940,136.91900,71.51120,21.627900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,"<a href=""statss.aspx?playerid=20123&position=O...",NYY,Yankees,152.77200,528.6340,663.5400,147.88600,82.76200,26.811600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,550.0,"<a href=""statss.aspx?playerid=24597&position=3...",TEX,Rangers,9.47552,23.6192,25.9712,5.39090,4.23297,0.779551,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,594.0,"<a href=""statss.aspx?playerid=22232&position=S...",SDP,Padres,14.58870,21.8830,23.7838,4.34710,2.96962,0.837784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,471.0,"<a href=""statss.aspx?playerid=sa917333&positio...",MIN,Twins,5.60117,19.3687,20.6742,4.12319,2.51295,0.872359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507,507.0,"<a href=""statss.aspx?playerid=sa3006891&positi...",TOR,Blue Jays,5.60248,19.4155,21.9682,4.67607,3.37489,1.004740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
final.sort_values('VORP', ascending=False).nlargest(200,['VORP']).to_csv("ATC.csv")

In [44]:
final[['PlayerName','Team','Pos','BestPos','G','GS','W','SV','value','VORP','ADP','rank','utilrank']].sort_values('VORP', ascending=False).nlargest(200,['VORP']).round(2).to_csv("ATC.csv", sep=",")